## Setup

In [1]:
import os
from dotenv import load_dotenv

# Load envirinment
load_dotenv()

# Get data from .env file
openai_api_key= os.getenv("OPENAI_API_KEY")
langsmith_api_key=os.getenv("LANGSMITH_API_KEY")
langsmith_tracing=os.getenv("LANGSMITH_TRACING")
langsmith_project=os.getenv("LANGSMITH_PROJECT")
langsmith_endpoint=os.getenv("LANGSMITH_ENDPOINT")

## Create Agent

In [2]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano-2025-08-07",
    system_prompt="You are an expert in digital marketing",
)

## No Steaming (invoke)

In [3]:
result = agent.invoke({"messages": [{"role": "user", "content": "Write a concise summary about digital marketing"}]})
print(result["messages"][1].content)

Digital marketing uses online channels to reach, engage, and convert target audiences. It combines multiple techniques to build awareness, nurture interest, drive conversions, and sustain loyalty.

Key components:
- Channels: search (SEO and paid), content marketing (blogs, video, podcasts), social media, email marketing, display/retargeting, affiliate and influencer marketing, and mobile/app marketing.
- Goals and funnel: awareness → consideration → conversion → retention → advocacy, guided by data and attribution.
- Measurement: metrics like impressions, clicks, CTR, conversions, CPA, ROAS, CAC, and customer lifetime value; ongoing testing and optimization.
- Benefits: scalable reach, measurable ROI, personalized targeting, faster experimentation, and tighter budget control.

How to approach: define audience and goals, create a cohesive content and channel plan, execute with consistent branding, test and optimize, and stay compliant with privacy regulations.

 Trends to watch: privac

## Stream mode = values

In [4]:
for step in agent.stream(
    {"messages":[{"role": "user", "content": "Write a concise summary about digital marketing"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Write a concise summary about digital marketing
================================== Ai Message ==================================

- Digital marketing is promoting products or services online to reach people where they spend time, using channels like search, social media, email, content, display ads, and mobile apps.
- It aims to attract, engage, convert, and retain customers across the buyer journey, aligning with business goals and measurable outcomes.
- Core channels include SEO/SEM, content marketing, social media, email marketing, and display/video advertising; success relies on targeting, messaging, and optimization.
- It’s data-driven: use analytics and experimentation (A/B tests) to measure metrics (traffic, leads, conversions, CAC, LTV, ROAS) and improve attribution across touchpoints.
- Benefits are scalability and measurable ROI, while challenges include privacy regulations, attribution complexi

## Stream mode = messages

## Messages stream data token by token - the lowest latency possible. 
## This is perfect for interactive applications like chatbots.

In [5]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write a short summary of skin care products for Facebook post"}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Here’s a short, Facebook-ready summary you can use:

Skincare basics: cleanse, hydrate, treat, moisturize, and protect. Start with a gentle cleanser, add a hydrating serum (like hyaluronic acid or vitamin C), seal in moisture with a moisturizer, and finish with broad‑spectrum sunscreen every day. Exfoliate 1–2 times a week and tailor products to your skin type. Patch‑test new ingredients and give your routine a few weeks to show results.

Want a version tailored to a specific skin type or audience? I can adjust the tone (friendly, professional, teen-friendly) or add hashtags.

### Tools can stream too!
### Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. 
### A "get_stream_writer" writer allows you to easily stream "custom" data from sources you create.

In [8]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

# create function
def get_weather(city: str) -> str:
    """Get weather for given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}"

# Create Agent
agent = create_agent(
    model="gpt-5-nano-2025-08-07",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in Portsmouth, United Kingdom"}]},
    stream_mode=["values", "custom"]
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in Portsmouth, United Kingdom', additional_kwargs={}, response_metadata={}, id='4ec1cdb5-5e2b-4c57-9000-edf19d2b1690')]})
('values', {'messages': [HumanMessage(content='What is the weather in Portsmouth, United Kingdom', additional_kwargs={}, response_metadata={}, id='4ec1cdb5-5e2b-4c57-9000-edf19d2b1690'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 133, 'total_tokens': 224, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnl9E4hr9dsnOwenX34h9Yl54YWn6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b2c55-4b12-7c91-be5f

In [9]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in Portsmouth, United Kingdom?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: Portsmouth, United Kingdom')
('custom', 'Acquired data for city: Portsmouth, United Kingdom')


### Modify the stream mode and the select to produce different results.

In [10]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in Portsmouth, United Kingdom?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: Portsmouth, United Kingdom
Acquired data for city: Portsmouth, United Kingdom
